# 📓 The GenAI Revolution Cookbook

**Title:** ChatGPT Prompts for Data Engineers That Boost Productivity

**Description:** Copy-paste ChatGPT prompts to automate pipelines, optimize SQL, debug Airflow and Spark, and enforce data quality for data engineers today.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Structured prompts are the difference between a model that returns generic advice and one that delivers production-ready code. When you ask for a data pipeline without specifying versions, schema, or constraints, the model defaults to surface-level reasoning. This guide shows you how to collapse 5–10 iterations into 1–2 by using version-pinned, constraint-first prompts that anchor the model to your production environment.

You'll learn one specific pattern: embedding schema, versions, and operational constraints directly into your prompt to force the model into production-aware reasoning. We'll cover the failure mode, why models behave this way, the fix, and when to use it.

```mermaid
%% Purpose: Visualize the difference between vague and structured prompts and their resulting outputs in data engineering tasks.

flowchart LR
    A[Vague prompt] --> B[Generic reasoning] %% Vague prompts lead to generic model reasoning
    B --> C[Surface-level advice]            %% Generic reasoning produces only surface-level advice
    D[Structured prompt] --> E[Constrained reasoning] %% Structured prompts lead to more constrained, relevant reasoning
    E --> F[Production-ready code]           %% Constrained reasoning results in production-ready code
```

## What Problem Are We Solving?

You ask the model to design a Spark pipeline. It returns code that uses deprecated APIs, ignores your schema, and assumes unlimited memory. You paste the schema. It rewrites the pipeline but still misses your SLA. You add the SLA. Now it suggests a different framework. After five rounds, you have something close, but you've burned 30 minutes and lost context.

The root cause: the model lacks anchors. Without explicit versions, schema, and constraints, it samples from a distribution of all possible pipelines across all environments. You get the average answer, not the one that fits your stack.

## What's Actually Happening Under the Hood

Language models generate text by predicting the next token based on context. When you provide a vague prompt, the model's context window contains only your high-level request. It retrieves patterns from training data that match "data pipeline" in general, leading to generic, framework-agnostic advice.

When you add schema, versions, and constraints upfront, you shift the retrieval distribution. The model now anchors to Spark 3.4, your exact column types, and your latency budget. This narrows the token prediction space, forcing the model to generate code that respects your environment. The schema acts as a structural prior, the version pins the API surface, and the constraints guide the reasoning path toward production-ready outputs.

Instruction hierarchy matters. Models prioritize information presented early and in structured blocks. If you bury the schema in a follow-up message, the model has already committed to a reasoning path. Frontloading constraints ensures the model's initial token predictions align with your requirements.

## How to Fix It: Prompt Patterns & Examples

The pattern is simple: pin versions, provide schema, and constrain output format at the top of your prompt. Use fenced blocks or delimiters to separate schema, sample data, and instructions. This prevents instruction bleed and keeps the model focused.

Here's a compact before/after comparison for a pipeline design task.

**Before (vague prompt):**

In [ ]:
Design a Spark pipeline to process user events.

**After (structured prompt):**

In [ ]:
Spark 3.4, Python 3.10. Schema: user_id (string), event_type (string), timestamp (long). Input: 10M rows/day, 500MB files. Output: Parquet, partitioned by date. Latency: <5 min. Return: PySpark code only.

The structured version anchors the model to your stack, schema, and SLA. The output will use Spark 3.4 APIs, respect your schema, and target your latency budget.

For SQL tuning, the same pattern applies. Instead of asking "optimize this query," provide the EXPLAIN plan, table schema, and performance target.

**Before:**

In [ ]:
Optimize this query: SELECT * FROM orders WHERE status = 'pending';

**After:**

In [ ]:
Postgres 14. Table: orders (id int, status varchar, created_at timestamp). Rows: 50M. EXPLAIN shows seq scan, 12s runtime. Target: <1s. Index exists on created_at. Return: optimized query + index suggestion.

The model now has the plan, the bottleneck, and the target. It will suggest an index on status and rewrite the query to avoid SELECT *.

Best practices for this pattern:

1. **Pin versions and workload targets at the top.** This sets the API surface and performance envelope before the model starts reasoning.
2. **Provide schema and sample plan.** Use fenced blocks to separate schema from instructions. This prevents the model from treating schema as part of the task description.
3. **Constrain output to code blocks and required artifacts.** Specify "Return: code only" or "Return: query + index" to avoid explanatory text that dilutes the output.

Use delimiters like triple backticks or XML-style tags to wrap schema, logs, and sample data. This keeps the model from blending instructions with context.

## Key Takeaways: When and Why to Use This

Use this pattern when:

- Outputs are generic or ignore your schema.
- The model suggests deprecated APIs or wrong frameworks.
- You need executable code with operational guidance.
- You want 1–2 turn convergence instead of 5–10 iterations.

Avoid this pattern when:

- You're exploring design options and want broad suggestions.
- The task is conceptual and doesn't require version-specific code.

Measure success by tracking acceptance rate of first draft, pass rate on schema validation, and whether the output hits your latency or cost budget. Aim for 80% acceptance on first try after applying this pattern.

For debugging workflows, add a 2-step loop: if the output misses a constraint, paste the EXPLAIN or error log into the next turn with the same structured prompt. If it still fails, add a schema snippet or sample row. This keeps the iteration tight and focused.

For structured JSON outputs in non-data-engineering contexts, see our guide on schema-compliant generation. For self-reflection loops that improve reasoning quality, see our guide on chain-of-thought prompting with validation steps.